# 环境安装与基本概念

使用一个工具，最先开始的一定就是安装了。本文件夹下主要针对pytorch进行介绍，也会补充一些tensorflow2的安装等。对于本repo，简单安装CPU版本的库即可，CPU版安装pytorch，[官网](https://pytorch.org/get-started/locally/)上查看命令可有：

```Shell
conda install pytorch torchvision cpuonly -c pytorch
```

tensorflow目前conda安装可能有版本不匹配的问题，可以在conda安装完毕之后，使用pip安装。接下来记录下如何安装GPU版的pytorch和tensorflow。

## ubuntu18.04安装pytorch1.2

### 版本搭配  

ubuntu18.04+cuda10.0+cudnn7.6.4+anaconda_python3.7+pytorch1.2

``` Shell
$ sudo lshw -C display | grep product
product: GP102 [GeForce GTX 1080 Ti]
```

### 驱动安装

先检查GPU的兼容性，[官网](https://developer.nvidia.com/cuda-gpus)上查看compute capability，满足大于3或者3.5的要求即可。

然后保证安装到Ubuntu上的包是最新的，因此

``` Shell
$ sudo apt update
$ sudo apt upgrade #will ask you Y/n if you want to upgrade
``` 

关于NVIDIA 的驱动：

Ubuntu预装了GPU的通用驱动。这些驱动不是优化的。因此需要找到适合自己的GPU的最新的NVIDIA驱动。有几种选择：

- Nvidia PPA：非常好的选择。通过使用PPA中包含的驱动达到开箱即用的效果。
- Ubuntu Default recommended driver：Ubuntu能指出自己的电脑需要的NVIDIA驱动
- Nouveau：NVIDIA驱动的开源实现版本。
- Official NVIDIA site：和PPA的一样，但是不会自动更新，并且有时在更新、卸载或者安装中会报错。

推荐的，也是比较好的方式是使用NVIDIA PPA来安装驱动。因为有最新的NVIDIA官方驱动，也在Ubuntu上测试过，且安装过程比较平滑。

``` Shell
$ sudo add-apt-repository ppa:graphics-drivers/ppa
$ sudo apt-get update
``` 

此语句将PPA库添加到Ubuntu的包系统内（Ubuntu是一种Debian Linux的发行版，因此使用的是Debian的dpkg包系统，该系统提供给Ubuntu应用来安装。高级包工具（APT）使得我们能很容易的在终端与dpkg交互）。

接下来需要决定安装哪个版本的driver。去[PPA库网站](https://launchpad.net/~graphics-drivers/+archive/ubuntu/ppa)上，
网页拉到最下面检查版本，发现很多版本都可以；也可以在[官网](https://www.nvidia.cn/Download/index.aspx?lang=cn)上检查，官网会给出一个版本；

在命令行输入：

``` Shell
$ ubuntu-drivers devices
``` 

可以看到给出了几个版本的驱动。

结合三者我决定选择nvidia-driver-410，因为410是对应cuda10.0的版本。

（**注意**：2019年10月24日看官网，支持的cuda版本已经为10.1，所以可以安装nvidia-driver-430，不过以内容下仍安装cuda10.0的安装写的）

``` Shell
$ sudo apt install nvidia-driver-410
``` 

安装完毕之后需要重启电脑，直接命令行reboot即可，以使用新的显卡驱动：

``` Shell
$ reboot
``` 

重启之后，在命令行键入：

``` Shell
$ nvidia-smi
``` 

查看是否已成功安装，出现一些信息基本上就是安装成功了。

如果安装的是430，问题也不大，即显示的信息里说CUDA版本是10.1的时候，
根据别人的[经验](https://zhuanlan.zhihu.com/p/73787970 )，装cuda-10.0也是可以的。 

为什么一定是10.0,？因为pytorch目前（2019年10月1日）支持的cuda是10.0。

为了保证驱动正常，所以有必要暂停其升级，使用以下命令：

``` Shell
$ sudo apt-mark hold nvidia-driver-410
nvidia-driver-410 set on hold.
``` 

To reverse this operation run:

``` Shell
$ sudo apt-mark unhold nvidia-driver-410
``` 

### 安装CUDA

首先check下gcc和g++的版本。

``` Shell
$ gcc --version
gcc (Ubuntu 7.3.0–16ubuntu3) 7.3.0
$ g++ --version
g++ (Ubuntu 7.3.0–16ubuntu3) 7.3.0
```

安装cuda10.0和9.0不太一样。直接从[官网](https://developer.nvidia.com/cuda-10.0-download-archive?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=1804&target_type=runfilelocal)下载
（用deb安装时我的报错了，不知道什么错误，有可能是因为cuda和driver的版本不完全匹配，所以用runfile安装了）

有提到要安装下kernel headers。（如果某个程序需要内核提供的一些功能，它就需要内核来编译程序，这个时候用的上kernel heads）。但几个也都没说。

``` code
$ sudo apt install linux-headers-$(uname -r)
```

执行命令即可安装，发现已经安装了。

按照官网提示安装即可，注意因为之前已经安装过驱动了，所以这里提示问是否安装驱动时（Install NVIDIA Accelerated Graphics Driver for Linux-x86_64 XXX.XX），选择no，其他的都可以选择yes或者默认值即可。

``` code
$ sudo ./cuda_10.0.130_410.48_linux.run
```

重启下看看自己没有黑屏。

然后配置环境变量。

``` code
$ vim ~/.bashrc #打开配置文件
$ export PATH=/usr/local/cuda-10.0/bin:$PATH
$ export LD_LIBRARY_PATH=/usr/local/cuda-10.0/lib64:$LD_LIBRARY_PATH
$ export CUDA_HOME=/usr/local/cuda-10.0
$ source ~/.bashrc
```
如果没有vim，安装即可，在命令行中输入：vim，没有安装的话会有提示，按照提示安装即可。

最后验证cuda是否安装成功：

``` code
$ cd /usr/local/cuda-10.0/samples/1_Utilities/deviceQuery
$ sudo make
$ ./deviceQuery
```

### 安装CUDNN

官网下载最新版的对应cuda10.0版本的CUDNN。

两种方法都有：

- 下载tar压缩包的：cudnn安装较容易只需要把文件解压后拷贝进cuda根目录即可。
- 下载三个deb文件的（the runtime library, the developer library, and the code samples library for Ubuntu 18.04）：

``` code
$ sudo dpkg -i libcudnn7_7.6.4.38-1+cuda10.0_amd64.deb
$ sudo dpkg -i libcudnn7-dev_7.6.4.38-1+cuda10.0_amd64.deb
$ sudo dpkg -i libcudnn7-doc_7.6.4.38-1+cuda10.0_amd64.deb 
```

安装完成验证是否安装成功：
Go to the MNIST example code: 

``` code
$ cd /usr/src/cudnn_samples_v7/mnistCUDNN/
```

Compile the MNIST example:

``` code
$ sudo make clean && sudo make
```

Run the MNIST example:

``` code
$ ./mnistCUDNN 
```

If your installation is successful, you should see Test passed!
现在已经不需要将gcc和g++降到6.X版本就可以执行。 

### 安装python

基础环境搭建好了之后，安装anaconda，现在（2019年10月1日）pytorch已经支持python3.7了。
下载linux下的安装包，执行：

``` code
$ sh Anaconda3-2019.07-Linux-x86_64.sh   #后边的文件名称是你的安装包的名称
```

然后一路按照默认的设置安装即可。
如果询问是否添加路径到环境变量或者是是否添加conda init，都选yes。

安装完成以后，重启终端。
先输入：

``` code
$ source ~/.bashrc
```

再输入：

``` code
$ python
```

即可看到安装成功的anaconda。

#### 安装pytorch

官网目前（2019年11月1日）已经是1.3版本了，对应cuda是10.1，这里安装的是cuda10.0，对应的pytorch版本是1.2，因此需要找[以前的版本](https://pytorch.org/get-started/previous-versions/)安装。版本对应上即可。使用conda安装的时候，可能会出现没有权限的情况，给自己的用户赋予conda和anaconda相关文件夹权限即可，命令如下（以我的为例，owen是用户名;/home/owen/.conda是我的conda的根目录）：

``` code
sudo chown -R owen /home/owen/.conda
sudo chown -R owen /home/owen/anaconda3
```

使用conda安装完毕后，检查是否安装成功：

``` code
$ python
$ import torch
```

能导入即成功。

可以先安装vscode，再测试下能不能使用GPU加速。
安装vscode直接去官网即可。然后下载对应的python插件即可使用。

``` code
$ sudo dpkg -i code_1.38.1-1568209190_amd64.deb
```

测试代码：

In [1]:
import torch as t
x = t.rand(5,3)
y = t.rand(5,3)
if t.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    print(x+y)

测试成功之后，就可以自己写程序试试了。


## Ubuntu18.04下安装TensorFlow-gpu

通过前面的步骤，应该已经安装好了cuda和cudnn了，这里就不再重复了。可以重新检查下自己的cuda和cudnn版本。

看看自己的显卡信息：

``` Shell
$ nvidia-smi
NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1 
...
``` 

上面这个cuda version可能不是你安装的，因为这里是说这个驱动匹配的cuda version

然后看看gcc和g++，比如我的一个版本：

``` Shell
$ gcc --version
gcc (Ubuntu 5.4.0-6ubuntu1~16.04.12) 5.4.0 20160609
$ g++ --version
g++ (Ubuntu 5.4.0-6ubuntu1~16.04.12) 5.4.0 20160609
``` 

可以看看cuda版本，比如：

```Shell
$ cat  /usr/local/cuda/version.txt
CUDA Version 10.0.130
```

还可以查看cudnn是否安装：

```Shell
$ cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2
cat: /usr/local/cuda/include/cudnn.h: No such file or directory
```

如果结果是上面这样的，说明cudnn没有安装好，可以看看cudnn是否已经下载：

```Shell
$ cat /usr/include/cudnn.h | grep CUDNN_MAJOR -A 2
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 5
#define CUDNN_PATCHLEVEL 1
...
```

如上所示，是7.5.1版本，说明cudnn已经下载了，但是还没往cuda里面放。如果你没有权限安装，那么就要通知管理员来安装了。

## win10下安装pytorch

现在pytorch和tensorflow都已经支持CUDA10.1了，因此安装cuda10.1及对应驱动，然后安装pytorch最新稳定版。首先如前所述，查看电脑的GPU是否足够支持pytorch和tensorflow： https://developer.nvidia.com/cuda-gpus#compute 

比如我的笔记本GeForce 940M	5.0 是支持的。查看自己的电脑的显卡可以通过右键“计算机”文件夹，然后点击“设备管理器”，然后找到“显示适配器”即可看到。

然后查看自己可下载的NVIDIA Driver： https://www.nvidia.com/download/index.aspx?lang=en-us ，不过在下载安装之前，先看看cuda10.1对应的driver版本是什么：https://developer.nvidia.com/cuda-toolkit-archive

选择10.1版之后可以看到要下载的文件名里有cuda10.1对应的nvidia driver的版本，比如我选择的是10.1 update2：https://developer.nvidia.com/cuda-10.1-download-archive-update2?target_os=Windows&target_arch=x86_64&target_version=10&target_type=exelocal ，可以看到名称是cuda_10.1.243_426.00_win10.exe ，也就是说驱动版本是426

接下来就去这个网站：https://www.geforce.com/drivers 上找自己电脑对应的驱动版本，在manual driver search里面选择自己的显卡和操作系统信息，然后点击“START SEARCH”即可搜出匹配的driver版本。可以看到我的搜出来最老的也是440.97了，也和426不一致，所以还是选择直接从安装系统时就安装的nvidia软件里去更新了，参考网站：https://www.geforce.cn/whats-new/articles/nvidia-update

右键桌面，选择“NVIDIA 控制面板”，即可进入NVIDIA的界面。然而我的“帮助”栏下面并没有更新项目。所以最后放弃，直接安装最新的版本了。

如果安装顺利，那么就继续。

根据 CUDA10.1 安装文档：https://docs.nvidia.com/cuda/archive/10.1/cuda-installation-guide-microsoft-windows/index.html

安装需要vs的支持，可以看到cuda10.1支持vs2019 16.x 及之前版本，所以先安装一个vs2019.

然后接下来就可以安装cuda了。

然后是cudnn的安装。

最后安装pytorch。

## win10下安装TensorFlow-gpu

### 版本搭配

win10+cuda10.0+cudnn7.6.4+anaconda_python3.7+tensorflow2.0

### 具体步骤

第一步，因为现在不论是TensorFlow2.0还是pytorch1.2都支持的是cuda10.0，而不完全支持cuda10.1，版本不匹配的话会有问题，因此cuda选择10.0。
如果电脑安装了visual studio 2019，那么安装cuda10.0时候会提示缺少visual studio 2017相关组件，因此需要首先安装visual studio 2017。
直接在visual studio官网上找，拉到页面最下面，找旧版本，不能下载的话，加入微软的开发者计划即可。这步下载安装过程可能比较长。

第二步: 去官网下载安装NVIDIA Driver
Select the appropriate version and click search

第三步: Install Anaconda (Python 3.7 version)

第四步: Update Anaconda
Open Anaconda Prompt to type the following command(s)

```Shell
conda update conda
conda update --all
```

第五步: 下载安装CUDA Tookit 10.0

Choose your version depending on your Operating System

第六步: Download cuDNN Download
Choose your version depending on your Operating System. Membership registration is required.

下载之后进行解压。

Add cuDNN into Environment PATH
Add the following path in your Environment. Subjected to changes in your installation path.

第七步: 安装TensorFlow

```Shell
$ pip install --upgrade pip
$ pip install tensorflow-gpu
```

接下来看看在一个已经安装过cuda和cudnn的win10电脑上如何使用pipenv安装tensorflow。

首先检查cuda和cudnn的版本：

打开cmd，输入：

```Shell
$ nvcc -V
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:12:52_Pacific_Daylight_Time_2019
Cuda compilation tools, release 10.1, V10.1.243
```

可以看到cuda的版本是10.1，接下来看看cudnn，因为是cuda10.1，所以进入cuda10.1的文件夹:C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v10.1\include，找到cudnn.h，文本编辑器打开就可以看到：

```Shell
...
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 6
#define CUDNN_PATCHLEVEL 4
...
```

如上图所示，我的是7.6.4版本。

接下来使用pipenv来安装tensorflow。首先建好一个项目，进入项目根目录。执行:

```Shell
$ pipenv isntall
```

这时候就会创建虚拟环境了。

然后在这里面安装tensorflow。根据现在最新的tensorflow版本情况，可以使用pip install tensorflow 方便的安装，这里使用pipenv install来安装：

```Shell
$ pipenv install tensorflow
```